In [255]:
import numpy as np
import torch
import transformers

In [256]:
tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'bert-base-uncased')
model = torch.hub.load('huggingface/pytorch-transformers', 'model', 'bert-base-uncased')

Using cache found in C:\Users\nazmievairat/.cache\torch\hub\huggingface_pytorch-transformers_main
Using cache found in C:\Users\nazmievairat/.cache\torch\hub\huggingface_pytorch-transformers_main


In [257]:
tokenizer

BertTokenizerFast(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [258]:
#tokenizer.vocab

In [303]:
text_1 = "Education is what remains after one has forgotten what one has learned in school"
#text_2 = "Educafion is wzat rempins aftqr onx hay porgotten uhat kne has lehrned in school"
text_2 = "Training is what remains before cat has remembered what nine has taught in kindergarten"
#text_2 = "Qducation is what remains after one has forgotten what one has learned in school"

text_tokens_1, _, _ = tokenizer(text_1, return_tensors='pt').values()
text_tokens_2, _, _ = tokenizer(text_2, return_tensors='pt').values()

In [304]:
tokenizer.tokenize(text_1, return_tensors='pt', add_special_tokens=False)

['education',
 'is',
 'what',
 'remains',
 'after',
 'one',
 'has',
 'forgotten',
 'what',
 'one',
 'has',
 'learned',
 'in',
 'school']

In [305]:
tokenizer.tokenize(text_2, return_tensors='pt', add_special_tokens=False)

['training',
 'is',
 'what',
 'remains',
 'before',
 'cat',
 'has',
 'remembered',
 'what',
 'nine',
 'has',
 'taught',
 'in',
 'kindergarten']

In [306]:
len(tokenizer.tokenize(text_1, add_special_tokens=True)), len(tokenizer.tokenize(text_2, add_special_tokens=True))

(16, 16)

In [307]:
a = tokenizer.pad(tokenizer(text_1, return_tensors='pt'), padding='max_length', max_length=max(text_tokens_1.shape[-1], text_tokens_2.shape[-1]))
b = tokenizer.pad(tokenizer(text_2, return_tensors='pt'), padding='max_length', max_length=max(text_tokens_1.shape[-1], text_tokens_2.shape[-1]))

In [308]:
a

{'input_ids': tensor([[ 101, 2495, 2003, 2054, 3464, 2044, 2028, 2038, 6404, 2054, 2028, 2038,
         4342, 1999, 2082,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [309]:
a_bert = model(**a).last_hidden_state[:, 1:-1]
b_bert = model(**b).last_hidden_state[:, 1:-1]

In [310]:
a_bert.shape, b_bert.shape

(torch.Size([1, 14, 768]), torch.Size([1, 14, 768]))

In [311]:
cos_sim =(torch.einsum('btn,btn->b', a_bert, b_bert) / torch.sqrt(torch.einsum('btn,btn->b', a_bert, a_bert) * torch.einsum('btn,btn->b', b_bert, b_bert)))
cos_sim.item()

0.6234679818153381